In [3]:
from function_registry import FunctionRegistry
import os
from langchain_cerebras import ChatCerebras
from dotenv import load_dotenv
# Set up the environment
from langchain.chat_models import init_chat_model

load_dotenv()
llm = ChatCerebras(
    model="llama-3.3-70b",
    api_key=os.getenv("CEREBRAS_API_KEY")
)

In [4]:
from parser import GoogleParser
import ai_functions

registry = FunctionRegistry(docstring_parser=GoogleParser())
registry.register_decorated_functions()
tools = registry.get_tools()
print(f"Tools: {tools}")
print(f"Registered {len(tools)} tools")

Tools: [{'type': 'function', 'function': {'name': 'calculate_m2_for_asset_components', 'description': 'calculate the square meters of an asset component', 'parameters': {'type': 'object', 'properties': {'asset_component_type': {'type': 'string', 'enum': ['IfcWindow', 'IfcDoor'], 'description': 'the ifc standard component type'}, 'floor': {'type': 'string', 'enum': ['1. Obergeschoss', '2. Obergeschoss', 'Erdgeschoss', 'Dachgeschoss', 'Untergeschoss'], 'description': 'floor of the building where the asset component is located'}}, 'required': []}}}]
Registered 1 tools


In [3]:
from pydantic import BaseModel, Field


class ResponseFormatter(BaseModel):
    """Always use this tool to structure your response to the user."""
    answer: str = Field(description="The answer to the user's question")
    sql_query: str = Field(description="SQl query to get the answer")


model_with_tools = llm.bind_tools([ResponseFormatter])

In [4]:
model_with_tools.invoke("How wide are windows in the Facility #3?")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '83603284c', 'function': {'arguments': '{"answer": "The width of the windows in Facility #3 is unknown", "sql_query": "SELECT width FROM facility_windows WHERE facility_id = 3"}', 'name': 'ResponseFormatter'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 278, 'total_tokens': 314, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'llama-3.3-70b', 'system_fingerprint': 'fp_a4e4dbcbd01d96252206', 'id': 'chatcmpl-b12fe2e3-7a00-4deb-9ee6-c673dcc4597c', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-60f7dbeb-283b-4325-9c66-c70e9e3fe0e9-0', tool_calls=[{'name': 'ResponseFormatter', 'args': {'answer': 'The width of the windows in Facility #3 is unknown', 'sql_query': 'SELECT width FROM facility_windows WHERE facility_id = 3'}, 'id': '83603284c', 'type': 'tool_call'}], usage_metadata={'input_tokens': 278, 'output_tokens'